# Logistic regression + PyTorch

In this notebook, we will introduce ourself to PyTorch machine learning framework by implement a linear regression model. Then we expand into a model with multiple linear transformation in the form of a simplest neural network. We exercise using a non-linear activation function and how that can impact the solution found by a model. 

### What we cover
1. Logistic regression with a linear model using PyTorch
2. Logistic regression with two layers perceptron


In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import plotly.graph_objs as go
layout = go.Layout(margin=dict(l=20,r=20,b=20,t=20),
                   template='plotly_dark',
                   width=500,height=400,
                  )
%matplotlib inline

## 1. Logistic regression: red v.s. blue

We start with PyTorch library by solving a logistic regression.

Our challenge is to separate the red and blue dots using data instances with 2 input features.

Here's a function for generating a dataset

In [ ]:
def generate_red_and_blue_normal(stat=2000,seed=123):
    
    np.random.seed(seed)
    
    data  = np.random.normal(1,1,stat*2).reshape(-1,2)
    label = np.zeros(stat).reshape(-1,1)

    mask = np.arange(stat)
    np.random.shuffle(mask)
    mask = mask[0:int(stat/2)]
    
    data [mask] -= 2
    label[mask]  = 1 
    
    return torch.Tensor(data),torch.Tensor(label)

... and always visualize to check it's not crazy :)

In [ ]:
import plotly.express as px
import plotly.graph_objs as go
data,label = generate_red_and_blue_normal()

blue=data[(label[:,0]==0)]
red =data[(label[:,0]==1)]

traces = [go.Scatter(x=blue[:,0],y=blue[:,1],opacity=0.7,mode='markers',name='blue'),
          go.Scatter(x= red[:,0],y= red[:,1],opacity=0.7,mode='markers',name='red'),
         ]
fig = go.Figure(traces,layout=layout)
fig.update_layout(xaxis_title='p0',yaxis_title='p1')
fig.show()

### 1.1 Building a model

1. Define a linear model $\sigma\left(\mathbf{w}\cdot\mathbf{x}+b\right)$ to classify red v.s. blue data points.
2. Use gradient descent to optimize the model parameters $\mathbf{w}$ and $b$ (three parameters).

#### 1.1.1 `torch.nn.Linear` module

Like many other ML libraries (e.g. `scikit-learn`), Pytorch comes with useful tools to build ML model. In particular, Pytorch provides algorithms in _modules_ and they can be easily combined to build a complex, large models such as a deep neural network. These modules come with capability to compute a derivative with respect to the input as well as parameters of the algorithm to be optimized. This enables gradient-based optimization for a composite model through a chain-rule (i.e. _back propagation_ of gradients).

One of the most frequently used is the `torch.nn.Linear` module.

In [ ]:
# Create a Linear module that takes 1 input and produces 1 output with a bias term
f=torch.nn.Linear(in_features=1,out_features=1,bias=True)

This module `f` performs a linear transformation:
$$ \text{output} = \mathbf{w}\cdot\mathbf{x} + b$$
where $\mathbf{x}\in\mathcal{R}^d$, $\mathbf{w}\in\mathcal{R}^{k\times d}$, and $b\in\mathcal{R}$. In the above construction, $d=1$ and $k=1$ are set by `in_features=1` and `out_features=1` respectively. The argument `bias=True` can be used to enable or remove the bias term in the model. 

The model parameters are randomly initialized (which we won't discuss here but it's typically sampled from a zero-centered normal distribution with _roughly_ a unit variance). Let's print them.

In [ ]:
print(f.weight)
print(f.bias)

All Pytorch modules are designed to take a _batch_ of input data. As such, the input data dimension must be 2 or larger where the first dimension specifies the size of input data. For instance, if we want to compute the model for an input data batch of size $n$ (i.e. $n$ data samples), the input should have the shape $(n,1)$.

In [ ]:
toy=torch.zeros(5).float().reshape(-1,1)
print('Input shape:',toy.shape)
print('Output:',f(toy))

You almost never need to do this, but if you want to force setting the model parameters, you can do this after disabling the gradient calculation:

In [ ]:
with torch.no_grad():
    f.weight[0,0]=1.
    f.bias[0]=2.
    toy=torch.arange(5).float().reshape(-1,1)
    print(f(toy))

#### 1.1.2 `torch.nn.Sigmoid`

Next, we need a logistic (i.e. sigmoid) function:

$$ \text{Sigmoid}(x) = \sigma(x) = \displaystyle{\frac{1}{1+\exp(-x)}}$$

... which is available as a Pytorch module:

In [ ]:
f=torch.nn.Sigmoid()

x = torch.arange(-5,5,0.01)

px.scatter(x=x,y=f(x)).show()

#### 1.1.3 Computation graph

Our model, $\sigma(\mathbf{w}\cdot\mathbf{x}+b)$, can be built by combining two modules (i.e.`torch.nn.Linear` and `torch.nn.Sigmoid`) in a sequence. This form the simplest computation graph: 
$$\text{input}\rightarrow\text{Linear}\rightarrow\text{Sigmoid}\rightarrow\text{output}$$

How do we build a computation graph instance by combining arbitrary number of operations? 

There are two typical ways. The first is to use `torch.nn.Sequential` container which can chain arbitrary number of modules to be called in a sequence:

In [ ]:
op0=torch.nn.Linear(in_features=2,out_features=1,bias=True)
op1=torch.nn.Sigmoid()

f = torch.nn.Sequential(op0,op1)
print( f(data) )

The second method is to define your own `torch.nn.Module` inherited class (in which you may use `torch.nn.Sequential` if wished)

In [ ]:
class model(torch.nn.Module):

    def __init__(self):
        super(model,self).__init__()
        self._op0 = torch.nn.Linear(in_features=2, out_features=1, bias=True)
        self._op1 = torch.nn.Sigmoid()
        
    def forward(self,x):
        return self._op1( self._op0(x) )

f = model()
print( f(data) )

Now that we have a way to design our model, let's move onto the optimization!

### 1.2 Training our model

Recalling from the lecture, below are the steps of gradient-descent optimization:

1. Define the model, loss, and optimization algorithm
2. Run a training loop
    2.1 Compute the loss
    2.2 Update model parameters using backpropagated gradients

Below is how these steps can be implemented using Pytorch


In [ ]:

criterion = torch.nn.BCELoss() # Step 1: Binary Cross-Entropy (BCE) loss
optimizer = torch.optim.SGD(params=f.parameters(),lr=0.001) # Step 1: SGD optimizer

for _ in range(10000):
    
    loss = criterion( f(data), label ) # Step 2.1: compute the loss
    optimizer.zero_grad() # Step 2.2: reset all gradients to zero
    loss.backward()       # Step 2.2: back propagate gradients from the loss
    optimizer.step()      # Step 2.2: inform the optimizer about the step


How is the model's classification performance? 

Let's build a confusion matrix:

In [ ]:
import plotly.figure_factory as ff
import time
test_data, test_label = generate_red_and_blue_normal(seed=int(time.time()))

def make_confusion_matrix(model,data,label):
    pred = f(test_data)
    blue = test_label == 0
    red  = test_label == 1
    tp = (red  & (pred >= 0.5)).sum().item() / red.sum().item()  # true positive
    tn = (blue & (pred <  0.5)).sum().item() / blue.sum().item() # true negative
    fp = (blue & (pred >= 0.5)).sum().item() / blue.sum().item() # false positive
    fn = (red  & (pred <  0.5)).sum().item() / red.sum().item()  # false negative

    cm = np.array([[tp, fn],[fp,tn]])
    return cm
    
cm = make_confusion_matrix(f,data,label)
cm_txt = cm.astype(np.str)
classes=['red','blue']

fig = ff.create_annotated_heatmap(z=cm, x=classes, y=classes, annotation_text=cm_txt, colorscale='Viridis')
fig.update_layout(width=500,height=500,
                  margin=dict(l=20,r=20,t=20,b=20),
                  xaxis_title='Prediction',yaxis_title='True label',
                 )
fig.show()

Let's visualize the decision boundary of the learned model

In [ ]:
blue=data[(label[:,0]==0)]
red =data[(label[:,0]==1)]

a,b = f._op0.weight.detach().numpy().reshape(-1)
c   = f._op0.bias.item()

xedges = np.array([data[:,0].min().item(),data[:,0].max().item()])
yedges = (-a/b)*xedges-c/b

traces = [go.Scatter(x=blue[:,0],y=blue[:,1],opacity=0.7,mode='markers',name='blue'),
          go.Scatter(x= red[:,0],y= red[:,1],opacity=0.7,mode='markers',name='red'),
          go.Scatter(x=xedges,y=yedges,name='boundary'),
         ]
fig = go.Figure(traces,layout=layout)
fig.update_layout(xaxis_title='p0',yaxis_title='p1')
fig.show()

### 1.2.1 Monitoring the training process

How can we improve our train procedure? One thing is to monitor the variables during the training. You can do this "by-hand" (i.e. writing your own code to keep the log of variable values over iterations), or use a tool like `Tensorboard`. 

Here is a simple example of `Tensorboard`:

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import time

# Create a handler to store the log for Tensorboard
writer = SummaryWriter(log_dir='aho')

for i in range(5):
    t0=time.time()
    writer.add_scalar('values/v0',i,i)
    writer.add_scalar('values/v1',i**2,i)
    writer.add_scalar('values/v2',i**3,i)
    writer.add_scalar('time_taken',time.time()-t0,i)

writer.add_graph(f,data)
writer.flush()


`Tensorboard` is most typically used for the purpose of a simple logging and visualization of the logged data. To visualize the log, we can run the `Tensorboard` (either from the command line or in-line within a notebook).

In [ ]:
%load_ext tensorboard

%tensorboard --logdir aho

Let's design a `train()` function with `Tensorboard` logging capability. For an analysis to be run later in this notebook, we design this function to also return the model parameter values. You might wonder: "why not storing model parameter values in the `Tensorboard` and read in later for analysis?" That is possible, but it does require an extra step of interpretting a binary file written by `Tensorboard`, which we won't cover in this notebook.

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from ipywidgets import IntProgress
from IPython.display import display
import time

def train(model,
          train_data, train_label,
          test_data=None, test_label=None,
          num_iter=10000,
          log_dir=None, store_cycle=100,
          optimizer='SGD', lr=0.01):
          
    if log_dir:
        writer = SummaryWriter(log_dir=log_dir)
    criterion = torch.nn.BCELoss()
    optimizer = getattr(torch.optim,optimizer)(model.parameters(),lr=lr)
    weights = []
    f = IntProgress(min=0,max=int(num_iter/400),bar_style='info')
    display(f)
    
    tstart = time.time()        
    for step in range(num_iter):

        weights.append(model._op0.weight.detach().numpy().reshape(-1).astype(np.float32))
        pred = model(data)
        loss = criterion( pred, label )
        optimizer.zero_grad()
        loss.backward()

        if log_dir and ((step+1)%store_cycle)==0:
            with torch.no_grad():
                # Monitor model accuracy & loss on the train dataset
                accuracy = (((pred < 0.5) & (label < 1)) | ((pred >= 0.5) & (label > 0))).sum().item() / len(pred)
                writer.add_scalar('acc/train',accuracy,step)
                writer.add_scalar('loss/train',loss,step)
                if (step+1)%100 == 0:
                    writer.add_histogram('score/train',pred,step)
                    
                # Monitor model parameters
                for i,w in enumerate(model._op0.weight.reshape(-1)):
                    writer.add_scalar('weights/p%02d' % i, w, step)
                for i,g in enumerate(model._op0.weight.grad.reshape(-1)):
                    writer.add_scalar('grads/g%02d' % i, g, step)
                    
                # Monitor model accuracy & loss on the test dataset (if provided)
                if test_data is not None and test_label is not None:
                    pred = model(test_data)
                    loss = criterion( pred, test_label )
                    accuracy = (((pred < 0.5) & (test_label < 1)) | ((pred >= 0.5) & (test_label > 0))).sum().item() / len(pred)
                    writer.add_scalar('acc/test',accuracy,step)
                    writer.add_scalar('loss/test',loss,step)
                    if (step+1)%100 == 0:
                        writer.add_histogram('score/test',pred,step)

        optimizer.step()      # Step 2.2: inform the optimizer about the step
        if step%400 == 0:
            f.value +=1
            
    if log_dir: writer.flush()
    print(time.time()-tstart,'[s]')
    return np.array(weights)

Now let's run the training loop

In [ ]:

torch.manual_seed(123)

f=model()
weights=train(f,data,label,test_data,test_label,10000,'aho/run00',100,'SGD')


In [ ]:
%tensorboard --logdir aho

### 1.2.2 Adam optimizer

As discussed in the lecture, there are multiple optimizer algorithms available. 
Let's try running another training with a popular (go-to) choice, `Adam` optimizer.


In [ ]:
torch.manual_seed(123)

f=model()
weights=train(f,data,label,test_data,test_label,10000,'aho/run01',100,'Adam')


Notice the log directory we just used is under the same directory `aho` as the last training using `SGD`. This allows us to compare two logs side-to-side. Go back to the `Tensorboard` cell and "reload". You should see two trainings available for visualization and comparison.

We can see `Adam` can flatten the loss curve faster than SGD reaching about the same level of the loss value!

Remember a difference depends on data, task, and your model to be optimized. There is no guaranteed "best solution". However, in practice, "Adam" is a very popular "default" choice for neural networks (later).

### 1.2.3 More fun

Here are two more "fun" visualization. The first is the sigmoid score contours overlaid with data points.

In [ ]:
def plot_sigmoid_surface(model,data):
    rangex = [torch.min(data[:,0]),torch.max(data[:,0])]
    rangey = [torch.min(data[:,1]),torch.max(data[:,1])]
    # expand the range by +/-20%
    rangex[0] -= (rangex[1]-rangex[0])*0.2
    rangex[1] += (rangex[1]-rangex[0])*0.2
    rangey[0] -= (rangey[1]-rangey[0])*0.2
    rangey[1] += (rangey[1]-rangey[0])*0.2
    xs = np.arange(rangex[0],rangex[1],(rangex[1]-rangex[0])/100.)
    ys = np.arange(rangey[0],rangey[1],(rangey[1]-rangey[0])/100.)
    grid = torch.Tensor(np.array(np.meshgrid(xs,ys,sparse=False)).reshape(2,-1))
    score = model(torch.swapaxes(grid,0,1)).reshape(len(xs),len(ys)).detach().numpy()
    
    blue = data[(label[:,0]<1)]
    red  = data[(label[:,0]>0)]
    
    fig = go.Figure(data=[go.Contour(x=xs,y=ys,z=score,name='loss'),
                          go.Scatter(x=blue[:,0],y=blue[:,1],opacity=0.7,mode='markers',name='blue'),
                          go.Scatter(x= red[:,0],y= red[:,1],opacity=0.7,mode='markers',name='red'),
                         ]
                   )
    fig.update_layout(
        autosize=False,
        margin=dict(l=20,r=20,b=20,t=20),
        width=600,height=400,
        xaxis_title='p0',yaxis_title='p1',
        legend_title='Data'
    )
    fig.show()
    
    
plot_sigmoid_surface(f,data)

Here's the history of updated parameter values overlaid on top of the loss surface

In [ ]:
def plot_convergence(model,param,data,label):
    range0 = [param[:,0].min(),param[:,0].max()]
    range1 = [param[:,1].min(),param[:,1].max()]
    # expand the range by +/-20%
    range0[0] -= (range0[1]-range0[0])*0.2
    range0[1] += (range0[1]-range0[0])*0.2
    range1[0] -= (range1[1]-range1[0])*0.2
    range1[1] += (range1[1]-range1[0])*0.2
    w0s = np.arange(range0[0],range0[1],(range0[1]-range0[0])/100.)
    w1s = np.arange(range1[0],range1[1],(range1[1]-range1[0])/100.)
    grid = torch.Tensor(np.array(np.meshgrid(w0s,w1s,sparse=False)).reshape(2,-1))
    
    loss_map = np.zeros(shape=(grid.shape[1]))
    with torch.no_grad():
        criterion = torch.nn.BCELoss()
        for i in range(grid.shape[1]):
            model._op0.weight[0,0] = grid[0,i]
            model._op0.weight[0,1] = grid[1,i]
            prediction = model(data)
            loss = criterion(prediction, label) 
            loss_map[i] = loss.mean()
    
    
    loss_map = loss_map.reshape(len(w0s),len(w1s))

    fig = go.Figure(data=[go.Contour(x=w0s,y=w1s,z=loss_map),
                          go.Scatter(x=param[:,0],y=param[:,1],mode='markers',
                                     marker=dict(color=np.log(np.arange(len(param))+1)),name='SGD'),
                         ]
                   )
    fig.update_layout(
        autosize=False,
        margin=dict(l=20,r=20,b=20,t=20),
        width=600,height=400,
        xaxis_title='w0',yaxis_title='w1',
        legend_title='Data'
    )
    fig.show()
    
plot_convergence(f,weights,data,label)



## 2. Logistic regression with two-layers MLP

Let's now make our problem a bit more challenging by modifying our dataset. 

A new data generator puts red and blue points along straight lines of the same slope but with different offsets.

A separation is hence straightforward, except we then re-distribute a half of red points somewhere else.

It's probably the simplest to see by eyes. Let's generate the sample and visualize!


In [ ]:

def generate_red_and_blue_irregular(a,b,seed=123,distort=False):
    np.random.seed(seed)
    num_unit=100
    x0 = np.zeros(shape=(num_unit*2),dtype=np.float32)
    x0[:num_unit] = np.arange(0,3,3.0/num_unit)
    x0[num_unit:] = np.arange(2,5,3.0/(num_unit))
    x1 = a*x0 + b + np.random.normal(scale=0.3,size=num_unit*2)
    x1[:num_unit] += 3
    x1[num_unit:] -= 3

    y = np.zeros(shape=(num_unit*2),dtype=np.int32)
    y[:num_unit] = 0
    y[num_unit:] = 1
    
    # if distort, shift some of label 0
    if distort:
        idx=np.arange(num_unit)
        np.random.shuffle(idx)
        idx=idx[:num_unit//2]
        x0[idx] = np.random.normal(size=len(idx),scale=0.3)+2.5
        x1[idx] = np.random.normal(size=len(idx),scale=0.3)
    
    
    data = np.column_stack([x0,x1,y])
    np.random.shuffle(data)

    return torch.Tensor(data[:,:2]),torch.Tensor(data[:,2].reshape(-1,1))
    


In [ ]:
# Use the same data generation function we used before
a,b=2.0,4.0
data,label=generate_red_and_blue_irregular(a,b,distort=True)
blue=data[(label[:,0]==0)]
red =data[(label[:,0]==1)]

traces = [go.Scatter(x=blue[:,0],y=blue[:,1],opacity=0.7,mode='markers',name='blue'),
          go.Scatter(x= red[:,0],y= red[:,1],opacity=0.7,mode='markers',name='red'),
         ]
fig = go.Figure(traces,layout=layout)
fig.update_layout(xaxis_title='p0',yaxis_title='p1')
fig.show()

By just look of it, it is clear we cannot separate two populations cleanly with a straight line. It is intuitively a difficult problem: I imagine different line slopes and offset but not very obvious what works the best immediately. A solution needs to wait till the later part of this notebook :)

### Exercise A

1. Optimize `model0` for with `Adam` with 10000 steps.
2. Visualize the resulting boundary overlaid on the data points

As expected, not so great :( 

### 2.1 Adding another `torch.nn.Linear`

During the lecture, we learned about adding a layer may help to linearize a non-linear dataset.

Let's build a new model that contains 2 linear layers. This means we have two instances of `torch.nn.Linear`. Make the second instance, which is our linear classifier, to take 2 inputs so we can visualize as a line in 2D space. That means the first instance of `torch.nn.Linear` should produce 2 output features (imagine the first instance contain 2 linear transformations).


In [ ]:
class model(torch.nn.Module):
    
    def __init__(self):
        super(model,self).__init__()
        # Define a linear model with a bias term
        self._op0  = torch.nn.Linear(2,2,bias=True)
        self._op1  = torch.nn.Linear(2,1,bias=True)
    def forward(self,x):
        return torch.sigmoid(self._op1(self._op0(x)))

### Exercise B
* Train this new model with the same dataset
* Visualize a linear decision boundary like we did before with the input data for the second linear layer

You should see the result did not improve much compared to a single linear transformation. That's because it's hard to distort the location of these data points, which is critical for whether a linear model works well or not, too much with only a linear transformation. 

That's right... what we have been lacking is a non-linearity.

So let's build another model with a non-linear activation function `torch.nn.LeakyReLU`.

### 2.3 Adding a non-linearity (Exercise C)

1. Design a model with `torch.nn.LeakyReLU` inserted between two linear layers
2. Train the model with `Adam` and 10000 iterations
3. Visualize the input data and two lines for the first layer
4. Visualize the input data and a line for the second layer


## Closing

By now, hopefully you feel familiar with a logistic regression using a linear model! and also getting used to how to define a model, optimize, and access the output using PyTorch. We will move onto a bit more complex models in the next notebooks.